In [1]:
"""
Created on Mon Jul 30 22:44:51 2018

For Small data-set, Single hidden layer NN with dropout regularizer of 0.5/0.8

@author: Shasha

"""

import pandas as pd
import numpy as np

from keras.wrappers.scikit_learn import KerasClassifier
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Flatten

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score


/home/test/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def single_nn_model():
    model = Sequential([
        Embedding(vocab_size + 1, 32, input_length=max_len),
        Flatten(),
        Dense(100, activation='relu'),
        Dropout(0.8),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [3]:
# Manual model Evaluation with K-fold
def cv_evaluate_nn_model(build_fn, X, y, nb_epoch=5, n_splits=5, batch_size=64, **kwargs):
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
    model = KerasClassifier(build_fn=build_fn, nb_epoch=nb_epoch, batch_size=batch_size, verbose=1)
    results = cross_val_score(model, X, y, cv=kfold)
    print('\nModel average accuracy: {:.2f}'.format(results.mean()))
    

In [4]:
train_small = "./dataset/training-data-small.txt"
test_small = "./dataset/test-data-small.txt"

In [5]:
#put the data in to a data frame and divide it in to Sample vs Label
train_small_df = pd.read_csv(train_small, sep = '\t', names = ["Label", "Sample"])

train_small_df_sample_len = train_small_df.Sample.str.split(",").apply(len)


In [6]:
#print (train_small_df_sample_len)

X_train, y = np.array(train_small_df.Sample), np.array(train_small_df.Label)

#print (X_train[:3])

test_small_df = pd.read_csv(test_small, sep = ' ', names = ["Sample"])

In [7]:
test_small_df_sample_len = test_small_df.Sample.str.split(",").apply(len)

X_test = np.array(test_small_df.Sample)

# concatenate both train, test set to build the vocabulary
X_all = np.concatenate((X_train, X_test))

In [8]:
tknzr = Tokenizer(lower = False, split = ',')
tknzr.fit_on_texts(X_all)
X_TrainSeqs = tknzr.texts_to_sequences(X_train)
X_TestSeqs = tknzr.texts_to_sequences(X_test)

#print (X_TestSeqs[:1])

In [9]:
max_len = train_small_df_sample_len.max() if train_small_df_sample_len.max() > test_small_df_sample_len.max() else test_small_df_sample_len.max()

X_TrainSeqs = sequence.pad_sequences(X_TrainSeqs, maxlen=max_len)

X_TestSeqs = sequence.pad_sequences(X_TestSeqs, maxlen=max_len)

In [10]:
print ("Train seqs data size ", X_TrainSeqs.shape)

vocab_size = len(tknzr.word_counts)
print ("Size of vocab : ", vocab_size)

Train seqs data size  (10000, 102)
Size of vocab :  127


In [11]:
# Model initialization & Training 
model = single_nn_model()

In [23]:
model.fit(X_TrainSeqs, y, batch_size=32, epochs=30)

Epoch 1/30
10000/10000 [==============================] - 5s 489us/step - loss: 0.1851 - acc: 0.9119
Epoch 2/30
10000/10000 [==============================] - 5s 478us/step - loss: 0.1864 - acc: 0.9102
Epoch 3/30
10000/10000 [==============================] - 5s 467us/step - loss: 0.1831 - acc: 0.9085
Epoch 4/30
10000/10000 [==============================] - 5s 491us/step - loss: 0.1715 - acc: 0.9186
Epoch 5/30
10000/10000 [==============================] - 5s 485us/step - loss: 0.1785 - acc: 0.9140
Epoch 6/30
10000/10000 [==============================] - 5s 489us/step - loss: 0.1731 - acc: 0.9183
Epoch 7/30
10000/10000 [==============================] - 5s 480us/step - loss: 0.1768 - acc: 0.9152
Epoch 8/30
10000/10000 [==============================] - 5s 472us/step - loss: 0.1730 - acc: 0.9156
Epoch 9/30
10000/10000 [==============================] - 5s 469us/step - loss: 0.1671 - acc: 0.9185
Epoch 10/30
10000/10000 [==============================] - 5s 482us/step - loss: 0.1694 - a

In [24]:
# Model Prediction
y_preds = model.predict(X_TestSeqs)
y_preds.shape

(100000, 1)

In [25]:
y_preds[-3]

array([0.99646854], dtype=float32)